Imports
---

In [ ]:
import os
import requests
import json
import plotly.offline as py
from plotly.graph_objs import *
py.offline.init_notebook_mode()
import pandas as pd

mapbox_access_token = 'pk.eyJ1IjoiY2hlbHNlYXBsb3RseSIsImEiOiJjaXFqeXVzdDkwMHFrZnRtOGtlMGtwcGs4In0.SLidkdBMEap9POJGIe1eGw'

Getting a log file
---

In [57]:
LOG_FOLDER = 'log'
MERGED_LOG = 'merged_log'
log_files = os.listdir(LOG_FOLDER)
API_URL = "http://freegeoip.net/json/{0}"

with open(MERGED_LOG, 'w') as outfile:
    for fname in log_files:
        with open(LOG_FOLDER + '/' + fname) as infile:
            outfile.write(infile.read())

Building a dict from a log file
---

In [58]:
ip_dic = {}
with open(MERGED_LOG) as logs:
    for line in logs:
        splited_log = line.split('"')
        ua = splited_log[-4]
        splited_log = line.split('- -')
        ip = splited_log[0].replace(' ', '')
        if ',' in ip:
            print line
        if ip in ip_dic:
            if ua not in ip_dic[ip]['ua']:
                ip_dic[ip]['ua'].append(ua)
        else:
            ip_dic[ip] = {}
            ip_dic[ip]['ua'] = [ua]

Formating the dict
--

In [59]:
for ip in ip_dic:
    google_bot_ua = ['Google bot' if 'bot' in ua else ua for ua in ip_dic[ip]['ua']]
    bing_bot_ua = ['Bing bot' if 'facebook' in ua else ua for ua in google_bot_ua]
    facebook_ua = ['Facebook' if 'facebook' in ua else ua for ua in bing_bot_ua]
    quora_ua = ['Quora' if 'quora' in ua else ua for ua in facebook_ua]
    ip_dic[ip]['ua'] = quora_ua       

Completing the dict with data coming from an API
----

In [71]:
for ip in ip_dic:
    url = API_URL.format(ip)
    response = requests.get(url)
    text_response = response.text
    json_response = json.loads(text_response)
    ip_dic[ip]['country'] = json_response['country_name']
    ip_dic[ip]['city'] = json_response['city']

Building a dict  of the hit per country
---

In [139]:
hit_per_coutry = {}
for ip in ip_dic:
    country = ip_dic[ip]['country']
    if country in hit_per_coutry:
        hit_per_coutry[country] += 1
    else:
        hit_per_coutry[country] = 1

Getting data for the map
---

In [140]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
df['GDP (BILLIONS)']  = 0
for index, row in df.iterrows():
    if row['COUNTRY'] in hit_per_coutry:
        df.loc[index, 'GDP (BILLIONS)'] = hit_per_coutry[row['COUNTRY']]

Defining the map and layout
---

In [138]:
data = [ dict(
        type = 'choropleth',
        locations = df['CODE'],
        z = df['GDP (BILLIONS)'],
        text = df['COUNTRY'],
        reversescale = True,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Number of hits'),
      ) ]

layout = dict(
    title = 'IP from around the world',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Merkador'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )